In [1]:
import rasa_nlu
import rasa_core
import spacy
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

In [6]:
training_data = load_data("nlu.md")
trainer = Trainer(config.load("config.yml"))

interpreter = trainer.train(training_data)

#storing trained data for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="restaurantbot")

c:\users\iishi\miniconda3\envs\chatbot\lib\site-packages\rasa_nlu\training_data\training_data.py:184: UserWarning: Entity 'no_of_people' has only 1 training examples! minimum is 2, training may fail.
  self.MIN_EXAMPLES_PER_ENTITY))
Epochs: 100%|█████████████████████████████████████████████████| 300/300 [00:03<00:00, 96.89it/s, loss=0.075, acc=1.000]


In [11]:
from rasa_nlu.test import run_evaluation

run_evaluation("nlu.md", model_directory)

INFO:tensorflow:Restoring parameters from C:\Users\Iishi\Documents\RASA-projects-master\RASA-projects-master\restaurant bot\./models/nlu\default\restaurantbot\component_6_EmbeddingIntentClassifier.ckpt


100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:00<00:00, 65.38it/s]


{'intent_evaluation': {'predictions': [{'text': 'hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9678974151611328},
   {'text': 'Hi',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9678974151611328},
   {'text': 'Hello',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9602600932121277},
   {'text': 'hello',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9602600932121277},
   {'text': 'good morning',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9719776511192322},
   {'text': 'good evening',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.9570649862289429},
   {'text': 'hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.957278311252594},
   {'text': 'Hey',
    'intent': 'greet',
    'predicted': 'greet',
    'confidence': 0.957278311252594},
   {'text': 'table for 2',
    'intent': 'book',
    'predicted': 'book',
    'confidence'

In [5]:
from rasa_core.policies import FallbackPolicy, KerasPolicy, MemoizationPolicy
from rasa_core.agent import Agent

In [7]:
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy()])

# loading our neatly defined training dialogues
training_data = agent.load_data('stories.md')

agent.train(training_data)

agent.persist('models/dialogue')

Processed Story Blocks: 100%|█████████████████████████████████████████████| 3/3 [00:00<00:00, 222.31it/s, # trackers=2]
Processed actions: 11it [00:00, 310.18it/s, # examples=11]


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 25)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7424      
_________________________________________________________________
dense (Dense)                (None, 15)                495       
_________________________________________________________________
activation (Activation)      (None, 15)                0         
Total params: 7,919
Trainable params: 7,919
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
21/21 [==============================] - 1s 64ms/sample - loss: 2.6917 - acc: 0.1429
Epoch 2/100
21/21 [==============================] - 0s 495us/sample - loss: 2.6059 - acc: 0.3333
Epoch 3/100
21/21 [==============================] - 0s 365us/sample - loss: 

In [ ]:
from rasa_core_sdk import Tracker
from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet
from rasa_core_sdk.executor import CollectingDispatcher

In [ ]:
class BookingConfirm(Action):
	def name(self):
		return "action_confirmation"
		
	def run(self, dispatcher, tracker, domain):
		no_of_people=tracker.slots['number_of_people']
		time=tracker.slots['time']
		date=tracker.slots['date']
		if date==None or no_of_people==None or time==None:
			response="Couldn't make reservations due to incomplete information received"
		else:
			response="""The table is booked for {} , {} at {} .""".format(no_of_people,date,time)				
		dispatcher.utter_message(response)
		return ''

In [ ]:
from rasa_core.agent import Agent
from rasa_core.utils import EndpointConfig
from rasa_core.interpreter import RasaNLUInterpreter

agent = Agent.load('models/dialogue',interpreter=RasaNLUInterpreter('./models/nlu/default/restaurantbot/'),
        action_endpoint=EndpointConfig(url="http://127.0.0.1:5055/webhook"))
print("Bot is ready to talk! Start conversation with 'hi'")
while True:
    st = input()
    if st == 'stop':
        break
    responses = agent.handle_text(st)
    for response in responses:
        print(response["text"])